In [9]:
import cv2
import numpy as np

cap = cv2.VideoCapture('original_video/2-1-1.mp4')

net = cv2.dnn.readNetFromCaffe('MobileNet-SSD/deploy.prototxt','MobileNet-SSD/mobilenet_iter_73000.caffemodel')

roi_startX, roi_startY, roi_endX, roi_endY = 180, 100, 730, 400

output_video = cv2.VideoWriter('output1_video.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 30, (int(cap.get(3)), int(cap.get(4))), True)

if net.empty():
    print('No data')
else:
    print('success model load')

success model load


In [10]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    roi = frame[roi_startY:roi_endY, roi_startX:roi_endX]
    blob = cv2.dnn.blobFromImage(roi, 0.007843, (300, 300), 127.5, swapRB=True)
    net.setInput(blob)
    detections = net.forward()

    cv2.rectangle(frame, (roi_startX, roi_startY), (roi_endX, roi_endY), (255, 0, 0), 2)

    p = 0
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.7:
            class_id = int(detections[0, 0, i, 1])

            # 計算偵測框在 ROI 中的位置
            box = detections[0, 0, i, 3:7] * np.array([
                roi.shape[1], roi.shape[0], roi.shape[1], roi.shape[0]])
            (x1, y1, x2, y2) = box.astype("int")

            # 加回偏移量到整張 frame 上
            x1 += roi_startX
            x2 += roi_startX
            y1 += roi_startY
            y2 += roi_startY

            # 畫框（黃色）
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)

            if class_id == 15:  # class_id 15 是 person
                p += 1

    text = f'People: {p}'
    cv2.putText(frame, text, (roi_startX, roi_startY - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    output_video.write(frame)

cap.release()
output_video.release()
print("Success video output")



Success video output
